In [29]:
def get_weather(city):
    return "33 degrees celcius."

FUNCTION_MAP = {
    "get_weather": get_weather
}

In [30]:
import openai, json

client = openai.OpenAI()

messages = []

In [31]:
TOOLS = [
    {
        "type":"function",
        "function": {
            "name": "get_weather",
            "description": "A function to get the weather of a city.",
            "parameters": {
                "type":"object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather of.",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [32]:
from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type":"function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )
    
        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: {function_name} with {arguments}")
            
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                arguments = {}

            function_to_run = FUNCTION_MAP.get(function_name)

            result = function_to_run(**arguments)

            print(f"Ran {function_name} with args {arguments} for a result of {result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": result,
                }
            )

        call_ai()

    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"AI: {message.content}")

def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages,
        tools=TOOLS,
    )
    process_ai_response(response.choices[0].message)

In [33]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()

User: My name is Theo
AI: Nice to meet you, Theo! How can I help you today? If you’d like, I can use your name during our chat to make things more personalized.
User: What is my name?
AI: Your name is Theo.

I’ll keep addressing you as Theo in this chat. If you’d like me to use a different name or nickname, just tell me.
User: What is the weather in Busan?
Calling function: get_weather with {"city":"Busan"}
Ran get_weather with args {'city': 'Busan'} for a result of 33 degrees celcius.
AI: Theo, the current weather in Busan is 33°C. It’s quite warm—you might want light clothing and to stay hydrated. Want me to pull humidity and wind details or check the forecast for the day?


In [34]:
messages

[{'role': 'user', 'content': 'My name is Theo'},
 {'role': 'assistant',
  'content': 'Nice to meet you, Theo! How can I help you today? If you’d like, I can use your name during our chat to make things more personalized.'},
 {'role': 'user', 'content': 'What is my name?'},
 {'role': 'assistant',
  'content': 'Your name is Theo.\n\nI’ll keep addressing you as Theo in this chat. If you’d like me to use a different name or nickname, just tell me.'},
 {'role': 'user', 'content': 'What is the weather in Busan?'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_J49iM56yW0yWodNvn00UhBio',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"Busan"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_J49iM56yW0yWodNvn00UhBio',
  'name': 'get_weather',
  'content': '33 degrees celcius.'},
 {'role': 'assistant',
  'content': 'Theo, the current weather in Busan is 33°C. It’s quite warm—you might want light clothing and to stay hydrated. Want me 